### Prevalence rates of Diabetes, with vearby Venue datacollected by County :
. use 110 counties, 52 categories onehot encoing first, then use mean() values\
. use CDC published Diabetes rates by coounty from 2009\
. the rate is dm_aap_2009 (aap: age-adjusted percentage)\
. examine data distribution/correlation, drop outliers, =>110 counties\
. use RandomForest and Decision Tree, both gives F1=0.857 (7/8 correct)\

In [1]:
import pandas as pd 
import numpy as np 

### 1. start with a list of counties, a list of categories, and a venue dataset

In [2]:
dfa = pd.read_csv("cdcfile.csv")
c = pd.read_csv("category_52.csv")
ds = pd.read_csv("ds_venues_120_30mi.csv")

In [3]:
print(dfa.shape)
dfa.head()

(110, 7)


,countyid,dm_2009,dm_aap_2009,ob_aap_2009,category_c,venue_c,aa_pop_2009
0,0500000US01073,60955.0,11.7,31.9,32,287,520982.905983
1,0500000US01117,11310.0,8.3,27.6,38,313,136265.060241
2,0500000US04021,23003.0,9.0,29.8,31,162,255588.888889
3,0500000US04025,14904.0,6.6,22.0,26,129,225818.181818
4,0500000US05007,11554.0,7.2,28.0,33,284,160472.222222


In [4]:
print(c.shape)
c.head()

(52, 2)


,category,v_count
0,American Restaurant,307
1,Athletics & Sports,467
2,BBQ Joint,2674
3,Bagel Shop,295
4,Bakery,343


In [5]:
print(ds.shape)
ds.head()

(56525, 7)


,countyid,venueid,venue,latitude,longitude,state,category
0,0500000US35043,5298b104498e725c6cf21cd6,"Zia Health Clinic, USPHS, IHS",35.498367,-106.730313,NM,Medical Center
1,0500000US35043,4efb651ebe7bb4785126a9c6,Jemez Health and Human Services,35.458076,-106.681770,NM,Medical Center
2,0500000US35043,4cc9df234650a35deb1b911e,El Pueblo Health Services,35.305300,-106.547200,NM,None
3,0500000US35043,51c35633498ee14993fc5a4a,Passport Health Rio Rancho Travel Clinic,35.238003,-106.655079,NM,Medical Center
4,0500000US35043,5cf676bfbb8d36002c75faed,UNM Health Sciences Rio Rancho Campus,35.312915,-106.683768,NM,General College & University


### 2. build feature set with venue categories - onehot encoing for each category

### i. join the category list to include only the categories of interest

In [6]:
ds = ds.merge(dfa[['countyid']], how = 'inner', on = 'countyid' )
dsv = ds.merge(c, how = 'inner', on = 'category' )
print(dsv.shape)

(31268, 8)


### ii. build the feature set

In [7]:
v = dsv[['countyid', 'category']]

# one hot encoding
v_onehot = pd.get_dummies(v[['category']], prefix="", prefix_sep="")
v_onehot['countyid'] = v['countyid'] 
fixed_columns = [v_onehot.columns[-1]] + list(v_onehot.columns[:-1])
v_onehot = v_onehot[fixed_columns]

vs = v_onehot.groupby(['countyid'], as_index = False).mean()
print(vs.shape)
vs.head()

(110, 53)


,countyid,American Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Breakfast Spot,Burger Joint,...,RV Park,Seafood Restaurant,Shopping Mall,Spa,Sporting Goods Shop,State / Provincial Park,Supermarket,Trail,Veterinarian,Yoga Studio
0,0500000US01073,0.020906,0.013937,0.108014,0.000000,0.020906,0.010453,0.017422,0.000000,0.027875,...,0.000000,0.017422,0.000000,0.003484,0.000000,0.000000,0.000000,0.000000,0.045296,0.034843
1,0500000US01117,0.012780,0.022364,0.102236,0.000000,0.012780,0.012780,0.012780,0.000000,0.038339,...,0.000000,0.015974,0.006390,0.003195,0.000000,0.012780,0.003195,0.000000,0.111821,0.028754
2,0500000US04021,0.006173,0.000000,0.061728,0.000000,0.018519,0.000000,0.006173,0.006173,0.043210,...,0.024691,0.000000,0.006173,0.000000,0.000000,0.006173,0.000000,0.018519,0.111111,0.006173
3,0500000US04025,0.015504,0.007752,0.062016,0.007752,0.000000,0.000000,0.023256,0.000000,0.046512,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038760,0.124031,0.054264
4,0500000US05007,0.014085,0.000000,0.105634,0.024648,0.024648,0.003521,0.007042,0.007042,0.063380,...,0.000000,0.000000,0.000000,0.000000,0.007042,0.000000,0.000000,0.000000,0.080986,0.049296


In [8]:
vs2 = vs
vs2.set_index('countyid', inplace = True)
print(vs2.shape)
vs2.head()

(110, 52)


,American Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Breakfast Spot,Burger Joint,Butcher,...,RV Park,Seafood Restaurant,Shopping Mall,Spa,Sporting Goods Shop,State / Provincial Park,Supermarket,Trail,Veterinarian,Yoga Studio
countyid,,,,,,,,,,,,,,,,,,,,,
0500000US01073,0.020906,0.013937,0.108014,0.000000,0.020906,0.010453,0.017422,0.000000,0.027875,0.003484,...,0.000000,0.017422,0.000000,0.003484,0.000000,0.000000,0.000000,0.000000,0.045296,0.034843
0500000US01117,0.012780,0.022364,0.102236,0.000000,0.012780,0.012780,0.012780,0.000000,0.038339,0.000000,...,0.000000,0.015974,0.006390,0.003195,0.000000,0.012780,0.003195,0.000000,0.111821,0.028754
0500000US04021,0.006173,0.000000,0.061728,0.000000,0.018519,0.000000,0.006173,0.006173,0.043210,0.012346,...,0.024691,0.000000,0.006173,0.000000,0.000000,0.006173,0.000000,0.018519,0.111111,0.006173
0500000US04025,0.015504,0.007752,0.062016,0.007752,0.000000,0.000000,0.023256,0.000000,0.046512,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038760,0.124031,0.054264
0500000US05007,0.014085,0.000000,0.105634,0.024648,0.024648,0.003521,0.007042,0.007042,0.063380,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.007042,0.000000,0.000000,0.000000,0.080986,0.049296


In [9]:
dv = vs2.merge(dfa[['countyid','dm_aap_2009']], how='inner', left_index= True, right_on='countyid')

## 3. Machine Learning Model - Decision Tree Based Classifier
Next, split the dataset into Training and Testing (15%). Use decisionTreeClassifier model to predict which group (high or low rate, highrate=1) a county is in, use features from onehot encoding from all revenue categories

### Random Forest Classifier Model

In [ ]:
# define target label variable
dfa = dfa[(dfa['dm_aap_2009'] <= 7.0) | (dfa['dm_aap_2009'] >= 9.3)]
dfa['highrate'] = dfa['dm_aap_2009'].apply(lambda x: 1 if (x>= 9.3) else 0)

In [13]:
dv2 = vs2.merge(dfa[['countyid','highrate']], how='inner', left_index= True, right_on='countyid')
dv2.shape

(51, 54)

In [14]:
dv2.head()

,American Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Breakfast Spot,Burger Joint,Butcher,...,Shopping Mall,Spa,Sporting Goods Shop,State / Provincial Park,Supermarket,Trail,Veterinarian,Yoga Studio,countyid,highrate
0,0.020906,0.013937,0.108014,0.000000,0.020906,0.010453,0.017422,0.000000,0.027875,0.003484,...,0.000000,0.003484,0.0,0.000000,0.0,0.000000,0.045296,0.034843,0500000US01073,1
3,0.015504,0.007752,0.062016,0.007752,0.000000,0.000000,0.023256,0.000000,0.046512,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.038760,0.124031,0.054264,0500000US04025,0
5,0.014388,0.000000,0.129496,0.014388,0.014388,0.003597,0.000000,0.014388,0.064748,0.003597,...,0.007194,0.000000,0.0,0.003597,0.0,0.000000,0.028777,0.061151,0500000US05119,1
6,0.011236,0.000000,0.174157,0.016854,0.011236,0.000000,0.005618,0.005618,0.022472,0.005618,...,0.000000,0.000000,0.0,0.005618,0.0,0.005618,0.061798,0.011236,0500000US05131,1
8,0.024911,0.000000,0.074733,0.003559,0.014235,0.003559,0.000000,0.010676,0.046263,0.003559,...,0.003559,0.003559,0.0,0.000000,0.0,0.000000,0.064057,0.067616,0500000US08001,0


In [15]:
data = dv2
data.set_index('countyid', inplace = True)
y = data['highrate'].values
X = data.drop(['highrate'], axis = 1)
X.head()
X = X.to_numpy()

,American Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Breakfast Spot,Burger Joint,Butcher,...,RV Park,Seafood Restaurant,Shopping Mall,Spa,Sporting Goods Shop,State / Provincial Park,Supermarket,Trail,Veterinarian,Yoga Studio
countyid,,,,,,,,,,,,,,,,,,,,,
0500000US01073,0.020906,0.013937,0.108014,0.000000,0.020906,0.010453,0.017422,0.000000,0.027875,0.003484,...,0.000000,0.017422,0.000000,0.003484,0.0,0.000000,0.0,0.000000,0.045296,0.034843
0500000US04025,0.015504,0.007752,0.062016,0.007752,0.000000,0.000000,0.023256,0.000000,0.046512,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.038760,0.124031,0.054264
0500000US05119,0.014388,0.000000,0.129496,0.014388,0.014388,0.003597,0.000000,0.014388,0.064748,0.003597,...,0.003597,0.007194,0.007194,0.000000,0.0,0.003597,0.0,0.000000,0.028777,0.061151
0500000US05131,0.011236,0.000000,0.174157,0.016854,0.011236,0.000000,0.005618,0.005618,0.022472,0.005618,...,0.000000,0.000000,0.000000,0.000000,0.0,0.005618,0.0,0.005618,0.061798,0.011236
0500000US08001,0.024911,0.000000,0.074733,0.003559,0.014235,0.003559,0.000000,0.010676,0.046263,0.003559,...,0.000000,0.000000,0.003559,0.003559,0.0,0.000000,0.0,0.000000,0.064057,0.067616


In [17]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [18]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state = 45)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (43, 52) (43,)
Test set: (8, 52) (8,)


In [19]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
clf = RandomForestClassifier(bootstrap=False, n_jobs=2, 
                             min_samples_leaf=5, min_samples_split= 6 , 
                             random_state=20, max_features= 50, 
                             n_estimators=50)

clf.fit(X_train, y_train)

# calculate probability of being a county with Highrate of DM prevalence
yhat_train = clf.predict_proba(X_train)[:,1] 
yhat_test = clf.predict_proba(X_test)[:,1]

In [37]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr_train, tpr_train, thresholds_train = roc_curve(y_train, yhat_train)
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, yhat_test)

auc_train = roc_auc_score(y_train, yhat_train)
auc_test = roc_auc_score(y_test, yhat_test)
print('Train AUC:%.3f'%auc_train)  
print('Test AUC:%.3f'%auc_test) 

Train AUC:0.976
Test AUC:0.867


In [38]:
# RandomForest produce probability scores, not 1/0
from sklearn import metrics
from sklearn.metrics import f1_score

yy = [0]*8
for i in range(0,8):
    if yhat_test[i] >= 0.55:        
        yy[i]=1
      
DS_f1 = f1_score(y_test, yy)
print("DecisionTrees / F1 Score: ", DS_f1)

DecisionTrees / F1 Score:  0.8571428571428571


In [39]:
# print the actual target labels and the predicted target values for the test set
print(y_test)
print(yhat_test)

[0 0 1 0 0 0 1 1]
[0.216      0.828      0.88628571 0.112      0.312      0.528
 0.742      0.67866667]


In [40]:
data.columns

Index(['American Restaurant', 'Athletics & Sports', 'BBQ Joint', 'Bagel Shop',
       'Bakery', 'Bar', 'Baseball Field', 'Breakfast Spot', 'Burger Joint',
       'Butcher', 'Café', 'Campground', 'Coffee Shop', 'Convenience Store',
       'Deli / Bodega', 'Dog Run', 'Donut Shop', 'Farmers Market',
       'Fast Food Restaurant', 'Flea Market', 'Food Court', 'Food Truck',
       'Furniture / Home Store', 'General Entertainment', 'Gourmet Shop',
       'Grocery Store', 'Gym', 'Health & Beauty Service', 'Home Service',
       'Hotel', 'Korean Restaurant', 'Market', 'Martial Arts Dojo',
       'Massage Studio', 'Miscellaneous Shop', 'Nightlife Spot', 'Non-Profit',
       'Other Great Outdoors', 'Park', 'Pet Service', 'Playground', 'Pool',
       'RV Park', 'Seafood Restaurant', 'Shopping Mall', 'Spa',
       'Sporting Goods Shop', 'State / Provincial Park', 'Supermarket',
       'Trail', 'Veterinarian', 'Yoga Studio', 'highrate'],
      dtype='object')

In [41]:
# print feature and gini values
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
    
feature_names = data.columns      

In [42]:
dd = pd.DataFrame(list(zip(feature_names,clf.feature_importances_ )))
dd.columns = ('category','gini')

In [45]:
# list features of the top 20 highest gini 
dd.sort_values('gini', ascending =False).head(20)

,category,gini
2,BBQ Joint,0.511513
18,Fast Food Restaurant,0.247135
25,Grocery Store,0.028079
37,Other Great Outdoors,0.027267
4,Bakery,0.022831
5,Bar,0.021499
7,Breakfast Spot,0.019749
9,Butcher,0.014721
46,Sporting Goods Shop,0.012295
1,Athletics & Sports,0.010758


In [46]:
dd = dd.sort_values('gini', ascending =False)
dd.to_csv("ds_final_gini.csv", index=False)

## Decision Tree Classifier

In [29]:
from sklearn.tree import DecisionTreeClassifier

In [30]:
dsTree = DecisionTreeClassifier(criterion="entropy", max_depth = 25) 

dsTree.fit(X_train, y_train)
yhat_train = dsTree.predict(X_train)
yhat_test = dsTree.predict(X_test)

print("DecisionTrees / Train set Accuracy: ", metrics.accuracy_score(y_train, yhat_train))
print("DecisionTrees / Test set Accuracy: ", metrics.accuracy_score(y_test, yhat_test))

fpr_train, tpr_train, thresholds_train = roc_curve(y_train, yhat_train)
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, yhat_test)

auc_train = roc_auc_score(y_train, yhat_train)
auc_test = roc_auc_score(y_test, yhat_test)
print('Train AUC:%.3f'%auc_train)  
print('Test AUC:%.3f'%auc_test) 

DecisionTrees / Train set Accuracy:  1.0
DecisionTrees / Test set Accuracy:  0.875
Train AUC:1.000
Test AUC:0.900


In [31]:
from sklearn import metrics
from sklearn.metrics import f1_score

DS_f1 = f1_score(y_test, yhat_test)
print("DecisionTrees / F1 Score: ", DS_f1)

DecisionTrees / F1 Score:  0.8571428571428571


In [32]:
# print the actual target labels and the predicted target values for the test set
print(y_test)
print(yhat_test)

[0 0 1 0 0 0 1 1]
[0 0 1 0 0 1 1 1]


## 4. Plot the Decision Tree Entropy Diagram
This will show which features (venue category) are chosen by the model. These are the one with the most differentiating power in the classification.

In [33]:
#!conda install -c conda-forge pydotplus -y
#!conda install -c conda-forge python-graphviz -y

In [34]:
from sklearn.externals.six import StringIO
import pydotplus

import matplotlib as mpl
import matplotlib.pyplot as plt

import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [35]:
filename = "DM_dsTree_110.png"
dot_data = StringIO()
featureNames = data.columns[0:52]
targetNames = data["highrate"].unique().tolist()
out=tree.export_graphviz(dsTree,feature_names=featureNames, out_file=dot_data, class_names= ['0','1'], filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')
plt.show()